# PHE SEIR Model

In this notebook we present how to use the `epimodels` module to set up an instantiation of the model built by Public Health England in collaboration with University of Cambridge.

In [1]:
# Load necessary libraries
import numpy as np
import epimodels as em
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'epimodels'

### Define setup matrices for the PHE Model

In [ ]:
# Populate the model
regions = ['London', 'Cornwall']
age_groups = ['0-10', '10-25']

# Initial state of the system
contact_data_matrix_0 = np.array([[1, 0], [0, 3]])
contact_data_matrix_1 = np.array([[10, 5.2], [0, 3]])

region_data_matrix_0_0 = np.array([[0.5, 0], [0, 6]])
region_data_matrix_0_1 = np.array([[1, 10], [1, 0]])
region_data_matrix_1_0 = np.array([[0.5, 1.2], [0.29, 6]])
region_data_matrix_1_1 = np.array([[0.85, 1], [0.9, 6]])

contacts_0 = em.ContactMatrix(age_groups, contact_data_matrix_0)
contacts_1 = em.ContactMatrix(age_groups, contact_data_matrix_1)
regional_0_0 = em.RegionMatrix(
    regions[0], age_groups, region_data_matrix_0_0)
regional_0_1 = em.RegionMatrix(
    regions[1], age_groups, region_data_matrix_0_1)
regional_1_0 = em.RegionMatrix(
    regions[0], age_groups, region_data_matrix_1_0)
regional_1_1 = em.RegionMatrix(
    regions[1], age_groups, region_data_matrix_1_1)

# Matrices contact
matrices_contact = [contacts_0, contacts_1]
time_changes_contact = [1, 3]
matrices_region = [
    [regional_0_0, regional_0_1],
    [regional_1_0, regional_1_1]]
time_changes_region = [1, 2]

### Set the parameters and initial conditions of the model and bundle everything together to simulate

In [ ]:
# Instantiate model
model = em.PheSEIRModel()

# Initial number of susceptibles
susceptibles = [[5, 6], [7, 8]]
dI = 4
initial_r = [0.5, 1]

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, 20.5, 0.5).tolist()

# List of initial conditions and parameters that characterise the model
parameters = [
    1, susceptibles, [[0.4, 0.2], [0.1, 0]], [[0.05, 0.2], [0.5, 0.32]],
    [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]],
    [[1]*len(times), [1]*len(times)], 4, dI, 0.5]

# Simulate using the ODE solver from scipy
scipy_method = 'RK45'

output_scipy_solver = model.simulate(
    parameters, times, matrices_contact, time_changes_contact,
    regions, initial_r, matrices_region, time_changes_region,
    method=scipy_method)

# Use different time steps for personalised solver
outputs_my_solver = []
time_steps = [0.5, 0.25, 0.05, 10**(-3)]

for ts in time_steps:
    # Update value of time step in parameters vector
    parameters[-1] = ts

    # Simulate using the 'homemade' discretised version of the ODE solver
    outputs_my_solver.append(model.simulate(
        parameters, times, matrices_contact, time_changes_contact,
        regions, initial_r, matrices_region, time_changes_region,
        method='my-solver'))

### Plot the comparments of the two methods against each other

In [ ]:
# Group outputs together
outputs = outputs_my_solver
outputs.append(output_scipy_solver)

# Trace names - represent the solver used for the simulation
trace_name = ['my-solver with delta_t = {}'.format(ts) for ts in time_steps]
trace_name.append('scipy-solver {}'.format(scipy_method))

# Compartment list - type and age
comparments = []
for n in model.output_names():
    comparments.extend(['{} for ages {}'.format(n, a) for a in age_groups])

for c, comparment in enumerate(comparments):
    # Plot (scatter plot for each comparment)
    fig = go.Figure()

    for o, out in enumerate(outputs):
        fig.add_trace(
            go.Scatter(
                y=out[:, c],
                x=times,
                mode='lines',
                name=trace_name[o]
            )
        )
    fig.update_layout(boxmode='group', title=comparment)
    fig.show()